In [7]:
import cv2
import numpy as np
import pandas as pd
import os
import pickle
import sys
sys.path.append('..')
from coordinate_transform import wgs_to_gcj02_to_mercator
from tqdm import tqdm


# 示例GPS点数据，假设数据包含 'longitude' 和 'latitude' 列
gps_path = "../../datasets/dataset_sz_1024size/GPS/bus_long50_traj_wgs84_coor/0_24_gps.pkl"
with open(gps_path, 'rb') as f:
    gps_df = pickle.load(f)

# tile's coordinate bounds
bounds_path = "../../datasets/dataset_sz_1024size/coordinates/0_24_gps.txt"
with open(bounds_path, 'r') as f:
    tile_bounds = eval(f.read())

# 创建一个空白的1024x1024二值图像
img_size = 1024
binary_image = np.zeros((img_size, img_size), dtype=np.uint8)

# 映射GPS点到图像坐标系
for index, row in tqdm(gps_df.iterrows()):
    # convert wgs84 to web mercator under gcj02
    row['lon'], row['lat'] = wgs_to_gcj02_to_mercator(row['lon'], row['lat'])

    if (tile_bounds['west'] <= row['lon'] <= tile_bounds['east'] and
            tile_bounds['south'] <= row['lat'] <= tile_bounds['north']):

        # 计算点在图像上的位置
        x = int((row['lon'] - tile_bounds['west']) / (tile_bounds['east'] - tile_bounds['west']) * (img_size - 1))
        y = int((tile_bounds['north'] - row['lat']) / (tile_bounds['north'] - tile_bounds['south']) * (img_size - 1))

        # 设置对应位置为白色（255）
        binary_image[y, x] = 255

# 保存图像
cv2.imwrite('save_images/binary_image.png', binary_image)


0it [00:00, ?it/s]

83264it [00:14, 5627.11it/s]


True